# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [3]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [4]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x23e3458bdc0>)

In [5]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [6]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'M1_Wildcard', 'ConfirmedCases',
       'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
       'StringencyLegacyIndex', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'Gove

In [7]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [8]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [9]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [10]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [11]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [12]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [13]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87064,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,78.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87065,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87066,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87067,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [14]:
# Set number of past days to use to make predictions
nb_lookback_days = 2

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [15]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [16]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.5,
                                                    random_state=301)

In [ ]:
# Création Random forest : 
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
import time

rf = RandomForestRegressor(n_estimators=10, criterion='mae', max_depth = None, 
                          min_samples_split=2, max_features= None)

start_time = time.time()
rf.fit(X_train, y_train)
interval = time.time() - start_time
print ('Total time in seconds:', interval)

In [17]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 140.7106897417153
Test MAE: 152.49541464435478


In [18]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.0010999660607081468
Day -6 NewCases 0.4395737128463477
Day -5 NewCases 0.21709357272996888
Day -4 NewCases 0.058996470378446894
Day -3 NewCases 0.06947070311451588
Day -2 NewCases 0.051910773903622094
Day -1 NewCases 0.2384190930581341
Day -26 C6_Stay at home requirements 4.314077585030425
Day -22 C2_Workplace closing 9.71354013447354
Day -17 C2_Workplace closing 5.767515924930496
Intercept 26.55512761911598


In [19]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [20]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [21]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 58.83064662471841
2020-08-02: 71.32284867347803
2020-08-03: 78.98708861551941
2020-08-04: 90.23987091036582
2020-08-05: 87.81079198105863
2020-08-06: 98.75906970570075
2020-08-07: 131.04183736458052
2020-08-08: 147.02193631910268
2020-08-09: 158.95064405735275
2020-08-10: 169.94035162547763
2020-08-11: 177.51587340468132
2020-08-12: 193.48225636298656
2020-08-13: 216.82128801942588
2020-08-14: 234.0388040158979
2020-08-15: 248.5583794521748
2020-08-16: 261.9657420386088
2020-08-17: 275.29735675692655
2020-08-18: 293.2898014228077
2020-08-19: 314.0742201473187
2020-08-20: 332.42691393545425
2020-08-21: 349.2299624107331
2020-08-22: 365.4979344992149
2020-08-23: 382.5309899042977
2020-08-24: 407.8898919660621
2020-08-25: 430.0819325457263
2020-08-26: 450.57044796445103
2020-08-27: 465.9978584772198
2020-08-28: 484.6446272952148
2020-08-29: 515.3477978671945
2020-08-30: 541.9000831891367
2020-08-31: 566.2609038448626

Predicting for Afghanistan__nan


2020-08-02: 2777.1793731666517
2020-08-03: 3002.2532845060327
2020-08-04: 2959.1075502670433
2020-08-05: 2639.3173861240666
2020-08-06: 1674.873575540922
2020-08-07: 2514.9605970299313
2020-08-08: 2970.0581444511813
2020-08-09: 3126.5711702281747
2020-08-10: 3100.8314389444704
2020-08-11: 2834.0040623697723
2020-08-12: 2564.854958718102
2020-08-13: 2961.2970849714084
2020-08-14: 3256.7404379252716
2020-08-15: 3377.0312143574165
2020-08-16: 3363.640932693805
2020-08-17: 3234.857010850697
2020-08-18: 3208.2398625175565
2020-08-19: 3439.483387568118
2020-08-20: 3639.91741226108
2020-08-21: 3740.556528944445
2020-08-22: 3755.7379719229025
2020-08-23: 3749.1725295270558
2020-08-24: 3805.5712797194724
2020-08-25: 3970.801255014097
2020-08-26: 4123.770478132545
2020-08-27: 4218.650336524838
2020-08-28: 4269.3769743450175
2020-08-29: 4316.145834391408
2020-08-30: 4406.200249245206
2020-08-31: 4545.121766420641

Predicting for Burundi__nan
2020-08-01: 56.71148451382387
2020-08-02: 66.5469936994

2020-08-19: 1620.9646491202138
2020-08-20: 1695.6945613816024
2020-08-21: 1723.7039689120152
2020-08-22: 1719.910282776772
2020-08-23: 1720.919942001573
2020-08-24: 1778.2787837839169
2020-08-25: 1867.4463489032764
2020-08-26: 1930.6039992369529
2020-08-27: 1957.2777312436401
2020-08-28: 1975.0767186240846
2020-08-29: 2003.2450517324155
2020-08-30: 2061.036255660064
2020-08-31: 2132.0571166671166

Predicting for Belarus__nan
2020-08-01: 1204.931488291726
2020-08-02: 1465.5176483110426
2020-08-03: 1541.6147793577647
2020-08-04: 1617.1783836778868
2020-08-05: 1378.394237964058
2020-08-06: 920.1094841658811
2020-08-07: 1392.696539460197
2020-08-08: 1601.8422914573061
2020-08-09: 1680.4555130833924
2020-08-10: 1697.3100711066822
2020-08-11: 1543.4490727419052
2020-08-12: 1426.3238101842044
2020-08-13: 1648.8576157957573
2020-08-14: 1795.6606181758484
2020-08-15: 1863.4445039117431
2020-08-16: 1869.868795482892
2020-08-17: 1805.204547507959
2020-08-18: 1794.379413864639
2020-08-19: 1922.448

2020-08-03: 131.84435339908464
2020-08-04: 108.80358523131275
2020-08-05: 113.84522997669325
2020-08-06: 144.68838809659547
2020-08-07: 241.5276659877805
2020-08-08: 292.5087160248432
2020-08-09: 231.620208142748
2020-08-10: 219.07386156960308
2020-08-11: 231.06335024071993
2020-08-12: 266.6348892366243
2020-08-13: 324.94398202192326
2020-08-14: 350.0832028573628
2020-08-15: 332.8496193024895
2020-08-16: 333.21599076339504
2020-08-17: 350.5740180508514
2020-08-18: 383.32544870480797
2020-08-19: 422.17165036966526
2020-08-20: 441.73434607335804
2020-08-21: 444.2460887609155
2020-08-22: 454.4017496652266
2020-08-23: 475.34492323667627
2020-08-24: 505.04300835143795
2020-08-25: 535.4232101353365
2020-08-26: 555.449425495507
2020-08-27: 568.4301806276973
2020-08-28: 585.4407857854446
2020-08-29: 609.0183426694277
2020-08-30: 637.2788551477993
2020-08-31: 664.9227551848761

Predicting for Central African Republic__nan
2020-08-01: 52.419979739204436
2020-08-02: 67.10765315697206
2020-08-03: 

Predicting for Congo__nan
2020-08-01: 57.56692588285915
2020-08-02: 89.78934111712333
2020-08-03: 126.4903002008672
2020-08-04: 89.61098653513523
2020-08-05: 88.22949245828299
2020-08-06: 102.93374398939838
2020-08-07: 138.27118566425233
2020-08-08: 167.3828835310932
2020-08-09: 185.26003194635186
2020-08-10: 177.88519157248015
2020-08-11: 183.7064639044048
2020-08-12: 201.80457250185364
2020-08-13: 228.83359431766775
2020-08-14: 252.90325546378034
2020-08-15: 267.9350584733194
2020-08-16: 273.755490829491
2020-08-17: 285.6700080023059
2020-08-18: 310.8742933359081
2020-08-19: 335.9196643546652
2020-08-20: 358.24385694948853
2020-08-21: 380.9855248265685
2020-08-22: 395.9549308130669
2020-08-23: 424.16186575092377
2020-08-24: 451.0903751940145
2020-08-25: 477.239952424775
2020-08-26: 512.2063757440012
2020-08-27: 538.7062926234125
2020-08-28: 562.973546105963
2020-08-29: 592.3683279182429
2020-08-30: 622.0851971578744
2020-08-31: 653.0607097295165

Predicting for Colombia__nan
2020-08-

...

2020-08-06: 4671.700899402452
2020-08-07: 7219.902188527033
2020-08-08: 8131.554594654367
2020-08-09: 7201.777588450921
2020-08-10: 6954.401301054314
2020-08-11: 7558.423436328475
2020-08-12: 6823.412929933199
2020-08-13: 7921.992640165173
2020-08-14: 8374.820324540688
2020-08-15: 8062.97814458015
2020-08-16: 8066.458216809095
2020-08-17: 8253.04463682857
2020-08-18: 8224.597611393718
2020-08-19: 8789.749573289451
2020-08-20: 9068.744100068881
2020-08-21: 9037.806874700549
2020-08-22: 9124.190346353833
2020-08-23: 9281.463944054381
2020-08-24: 9448.147199665156
2020-08-25: 9809.190371921342
2020-08-26: 10036.488876933117
2020-08-27: 10131.428313339939
2020-08-28: 10272.85953643554
2020-08-29: 10454.012901407112
2020-08-30: 10676.37317088428
2020-08-31: 10962.451812374347

Predicting for United States__Minnesota
2020-08-01: 6186.2217908168805
2020-08-02: 6743.681821524891
2020-08-03: 6673.6802285716585
2020-08-04: 6972.658166745846
2020-08-05: 5689.109965029104
2020-08-06: 3980.45761627

2020-08-15: 4651.257341480736
2020-08-16: 4556.902218543754
2020-08-17: 4356.862592269107
2020-08-18: 4384.352085223666
2020-08-19: 4737.153017306558
2020-08-20: 4991.012941440548
2020-08-21: 5092.746026091134
2020-08-22: 5071.539255188205
2020-08-23: 5037.859420121641
2020-08-24: 5139.227389407225
2020-08-25: 5376.39935705968
2020-08-26: 5568.680703024176
2020-08-27: 5667.971636031294
2020-08-28: 5707.561107646672
2020-08-29: 5756.682729738115
2020-08-30: 5884.701271853238
2020-08-31: 6072.490509088854

Predicting for United States__New Mexico
2020-08-01: 2737.948975465311
2020-08-02: 2828.0603397357145
2020-08-03: 2565.860570583326
2020-08-04: 2570.492415352447
2020-08-05: 2147.332801033023
2020-08-06: 1638.0316929579453
2020-08-07: 2700.106829385356
2020-08-08: 2883.464025314793
2020-08-09: 2807.8151313610556
2020-08-10: 2752.915737214704
2020-08-11: 2514.7255734027226
2020-08-12: 2466.8775116872107
2020-08-13: 2940.933833083181
2020-08-14: 3097.0323340742157
2020-08-15: 3096.511526

2020-08-12: 1532.542018578823
2020-08-13: 1839.9287971049407
2020-08-14: 1990.6150301231532
2020-08-15: 1985.8913681312247
2020-08-16: 1926.6881813564983
2020-08-17: 1875.0770791166092
2020-08-18: 1927.9398232595
2020-08-19: 2101.2874749454763
2020-08-20: 2203.833117475684
2020-08-21: 2223.1448272260573
2020-08-22: 2210.99983071059
2020-08-23: 2215.181691598962
2020-08-24: 2283.752663209939
2020-08-25: 2399.133194965614
2020-08-26: 2479.2351872180957
2020-08-27: 2510.983951316491
2020-08-28: 2530.3631073652637
2020-08-29: 2565.7145511820286
2020-08-30: 2637.275309473208
2020-08-31: 2727.574532144269

Predicting for United States__South Dakota
2020-08-01: 978.5313692485526
2020-08-02: 1221.8534587785316
2020-08-03: 1128.0478021315357
2020-08-04: 996.3389518955786
2020-08-05: 834.5036066477986
2020-08-06: 665.0921177150578
2020-08-07: 1083.7151325091686
2020-08-08: 1243.3851384902691
2020-08-09: 1212.3077121768824
2020-08-10: 1139.1418646040288
2020-08-11: 1047.8125599664577
2020-08-12: 

2020-08-13: 6065.871243607649
2020-08-14: 6491.471958123582
2020-08-15: 6348.725982735381
2020-08-16: 6005.1631402236835
2020-08-17: 5807.505698872053
2020-08-18: 6061.429614424781
2020-08-19: 6670.112424649934
2020-08-20: 6951.755218381215
2020-08-21: 6919.616098144852
2020-08-22: 6789.751681068897
2020-08-23: 6780.459545411834
2020-08-24: 7029.427951714276
2020-08-25: 7406.369655223323
2020-08-26: 7618.351926059033
2020-08-27: 7658.883166640768
2020-08-28: 7661.287815526995
2020-08-29: 7750.751941046445
2020-08-30: 7978.790030273762
2020-08-31: 8252.348578931982

Predicting for United States__West Virginia
2020-08-01: 944.5232004803574
2020-08-02: 1091.0930242391287
2020-08-03: 1081.029251873499
2020-08-04: 956.7846684009315
2020-08-05: 746.9565228976969
2020-08-06: 623.6404622204207
2020-08-07: 1020.4294902183832
2020-08-08: 1150.0056423616884
2020-08-09: 1149.3154746754412
2020-08-10: 1075.9892598683205
2020-08-11: 971.7392356399649
2020-08-12: 982.3772532653348
2020-08-13: 1176.77

2020-08-27: 444.77041853320804
2020-08-28: 461.7595235454223
2020-08-29: 480.9293160491151
2020-08-30: 501.35741751775276
2020-08-31: 522.2219490701939

Predicting for South Africa__nan
2020-08-01: 2352.9530736819106
2020-08-02: 2707.796992781089
2020-08-03: 2894.391731333827
2020-08-04: 2704.097581188429
2020-08-05: 2173.294272101071
2020-08-06: 1583.324792231194
2020-08-07: 2521.7326858524516
2020-08-08: 2865.8160561665004
2020-08-09: 2965.36528633069
2020-08-10: 2838.670051756688
2020-08-11: 2531.2825529187944
2020-08-12: 2422.4374978771875
2020-08-13: 2864.1502655359477
2020-08-14: 3108.8775141146157
2020-08-15: 3181.0914780275943
2020-08-16: 3112.9630249348625
2020-08-17: 2984.6410654556307
2020-08-18: 3023.4418362421425
2020-08-19: 3272.7331342053626
2020-08-20: 3444.987884170018
2020-08-21: 3511.344886400539
2020-08-22: 3497.9891735337455
2020-08-23: 3486.5785641303814
2020-08-24: 3568.971229050586
2020-08-25: 3738.0298581302136
2020-08-26: 3871.4316378265207
2020-08-27: 3943.52

In [22]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,58.830647
214,Aruba,NaN,2020-08-02,71.322849
215,Aruba,NaN,2020-08-03,78.987089
216,Aruba,NaN,2020-08-04,90.239871
217,Aruba,NaN,2020-08-05,87.810792


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [23]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [24]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,58.83064662471841
Aruba,,2020-08-02,71.32284867347803
Aruba,,2020-08-03,78.98708861551941
Aruba,,2020-08-04,90.23987091036582
Afghanistan,,2020-08-01,149.7731977908312
Afghanistan,,2020-08-02,287.29069834944903
Afghanistan,,2020-08-03,277.525581745693
Afghanistan,,2020-08-04,264.8071924260645
Angola,,2020-08-01,198.69836835573278


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [27]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(current_dir))))
sys.path.insert(0, parent_dir)

In [28]:
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [29]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [30]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
Wall time: 3.83 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [31]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-01
End date: 2021-05-30


In [33]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [34]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-01 to 2021-05-30...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
Wall time: 5min 17s
